## Install dependencies and import

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
!pip install numpy
!pip install torch
!pip install matplotlib
!pip install scipy
!pip install torchvision
!pip install tqdm
!pip install visdom
!pip install nibabel
!pip install scikit-image
!pip install h5py
!pip install pandas
!pip install dominate
!pip install pydicom
!pip install opencv-python
!pip install scikit-learn
!pip install tensorflow
!pip install https://github.com/ozan-oktay/torchsample/tarball/master#egg=torchsample-0.1.3

In [0]:
import os
import shutil
import zipfile
import json
import collections
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pydicom
import numpy as np
import pandas as pd
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numbers
import PIL

## Setup dataset

In [0]:
# local machine directories
DATA_DIR = "/content/data/"
DATA_MASKS_DIR = DATA_DIR + "masks"
DATA_DICOM_DIR = DATA_DIR + "input"

# google drive directories
DATA_ZIP_FILE_PATH = "/content/gdrive/My Drive/mlmi/dataset/input.zip"
MASKS_DIR = "/content/gdrive/My Drive/mlmi/dataset/masks/"
TENSORBOARD_LOGS_DIR = "/content/gdrive/My Drive/mlmi/results/tensorboard/"
MODEL_SAVE_DIR = "/content/gdrive/My Drive/mlmi/results/model/"


Copy masks from gdrive

In [0]:
# create data directory locally
if os.path.exists(DATA_DIR) == False:
  os.makedirs(DATA_DIR)
  print("Create directory: " + DATA_DIR)

In [0]:
if os.path.exists(DATA_MASKS_DIR):
  shutil.rmtree(DATA_MASKS_DIR)
  print("Deleted old masks directory: " + DATA_MASKS_DIR)

!cp -r "$MASKS_DIR" $DATA_DIR
%ls $DATA_MASKS_DIR

In [0]:
#### uncomment this code to reduce the train set size
# includeImagesCounts = 50
# trainSetCsv = pd.read_csv(DATA_MASKS_DIR + '/simm_DS_train.csv')
# tsReduced = trainSetCsv[:includeImagesCounts]
# tsReduced[:5]

# tsReduced.to_csv(DATA_MASKS_DIR + '/simm_DS_train.csv', index = None, header=True)

Extract DICOM files from zip

In [0]:
# only extract if input folder doesn't exists
if os.path.exists(DATA_DICOM_DIR) == False:
  #!unzip "$DATA_ZIP_FILE_PATH" -d "$DATA_DIR"
  zip_ref = zipfile.ZipFile(DATA_ZIP_FILE_PATH, 'r')
  zip_ref.extractall(DATA_DIR)
  zip_ref.close()
  print("Extracted dicom zip file to directory: " + DATA_DICOM_DIR)

%ls $DATA_DICOM_DIR
#%ls "$DATA_DICOM_DIR/siim/dicom-images-train"

## Model Definitions

In [0]:
def init_weights(net, init_type='normal'):
    #print('initialization method [%s]' % init_type)
    if init_type == 'normal':
        net.apply(weights_init_normal)
    elif init_type == 'xavier':
        net.apply(weights_init_xavier)
    elif init_type == 'kaiming':
        net.apply(weights_init_kaiming)
    elif init_type == 'orthogonal':
        net.apply(weights_init_orthogonal)
    else:
        raise NotImplementedError('initialization method [%s] is not implemented' % init_type)

def weights_init_kaiming(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        nn.init.normal(m.weight.data, 1.0, 0.02)
        nn.init.constant(m.bias.data, 0.0)
        
class unetConv2(nn.Module):
    def __init__(self, in_size, out_size, is_batchnorm, n=2, ks=3, stride=1, padding=1):
        super(unetConv2, self).__init__()
        self.n = n
        self.ks = ks
        self.stride = stride
        self.padding = padding
        s = stride
        p = padding
        if is_batchnorm:
            for i in range(1, n+1):
                conv = nn.Sequential(nn.Conv2d(in_size, out_size, ks, s, p),
                                     nn.BatchNorm2d(out_size),
                                     nn.ReLU(inplace=True),)
                setattr(self, 'conv%d'%i, conv)
                in_size = out_size

        else:
            for i in range(1, n+1):
                conv = nn.Sequential(nn.Conv2d(in_size, out_size, ks, s, p),
                                     nn.ReLU(inplace=True),)
                setattr(self, 'conv%d'%i, conv)
                in_size = out_size

        # initialise the blocks
        for m in self.children():
            init_weights(m, init_type='kaiming')

    def forward(self, inputs):
        x = inputs
        for i in range(1, self.n+1):
            conv = getattr(self, 'conv%d'%i)
            x = conv(x)

        return x

class unetUp2(nn.Module):
    def __init__(self, in_size, out_size, is_deconv, is_batchnorm=True):
        super(unetUp2, self).__init__()
        self.conv = unetConv2(in_size + out_size, out_size, is_batchnorm)
        if is_deconv:
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=4, stride=2, padding=1)
        else:
            self.up = nn.UpsamplingBilinear2d(scale_factor=2)

        # initialise the blocks
        for m in self.children():
            if m.__class__.__name__.find('unetConv2') != -1: continue
            init_weights(m, init_type='kaiming')

    def forward(self, inputs1, inputs2):
        outputs2 = self.up(inputs2)
        # offset = outputs2.size()[1] - inputs1.size()[1]
        # padding = [0, 0, 0, 0, offset // 2, offset // 2]
        # outputs1 = F.pad(inputs1, padding)
        outputs1 = inputs1
        cat = torch.cat([outputs1, outputs2], 1)
        return self.conv(cat)

class UnetGridGatingSignal2(nn.Module):
    def __init__(self, in_size, out_size, ks=1, is_batchnorm=True):
        super(UnetGridGatingSignal2, self).__init__()

        if is_batchnorm:
            self.conv1 = nn.Sequential(nn.Conv2d(in_size, out_size, ks),
                                       nn.BatchNorm2d(out_size),
                                       nn.ReLU(inplace=True),
                                       )
        else:
            self.conv1 = nn.Sequential(nn.Conv2d(in_size, out_size, ks),
                                       nn.ReLU(inplace=True),
                                       )

        # initialise the blocks
        for m in self.children():
            init_weights(m, init_type='kaiming')

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        return outputs

class UnetDsv2(nn.Module):
    def __init__(self, in_size, out_size, scale_factor):
        super(UnetDsv2, self).__init__()
        self.dsv = nn.Sequential(nn.Conv2d(in_size, out_size, kernel_size=1, stride=1, padding=0),
                                 nn.Upsample(scale_factor=scale_factor, mode='bilinear'), )

    def forward(self, input):
        return self.dsv(input)

class _GridAttentionBlockND(nn.Module):
    def __init__(self, in_channels, gating_channels, inter_channels=None, dimension=3, mode='concatenation',
                 sub_sample_factor=(2,2,2)):
        super(_GridAttentionBlockND, self).__init__()

        assert dimension in [2, 3]
        assert mode in ['concatenation', 'concatenation_debug', 'concatenation_residual']

        # Downsampling rate for the input featuremap
        if isinstance(sub_sample_factor, tuple): self.sub_sample_factor = sub_sample_factor
        elif isinstance(sub_sample_factor, list): self.sub_sample_factor = tuple(sub_sample_factor)
        else: self.sub_sample_factor = tuple([sub_sample_factor]) * dimension

        # Default parameter set
        self.mode = mode
        self.dimension = dimension
        self.sub_sample_kernel_size = self.sub_sample_factor

        # Number of channels (pixel dimensions)
        self.in_channels = in_channels
        self.gating_channels = gating_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        if dimension == 3:
            conv_nd = nn.Conv3d
            bn = nn.BatchNorm3d
            self.upsample_mode = 'trilinear'
        elif dimension == 2:
            conv_nd = nn.Conv2d
            bn = nn.BatchNorm2d
            self.upsample_mode = 'bilinear'
        else:
            raise NotImplemented

        # Output transform
        self.W = nn.Sequential(
            conv_nd(in_channels=self.in_channels, out_channels=self.in_channels, kernel_size=1, stride=1, padding=0),
            bn(self.in_channels),
        )

        # Theta^T * x_ij + Phi^T * gating_signal + bias
        self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=self.sub_sample_kernel_size, stride=self.sub_sample_factor, padding=0, bias=False)
        self.phi = conv_nd(in_channels=self.gating_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0, bias=True)
        self.psi = conv_nd(in_channels=self.inter_channels, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)

        # Initialise weights
        for m in self.children():
            init_weights(m, init_type='kaiming')

        # Define the operation
        if mode == 'concatenation':
            self.operation_function = self._concatenation
        elif mode == 'concatenation_debug':
            self.operation_function = self._concatenation_debug
        elif mode == 'concatenation_residual':
            self.operation_function = self._concatenation_residual
        else:
            raise NotImplementedError('Unknown operation function.')


    def forward(self, x, g):
        '''
        :param x: (b, c, t, h, w)
        :param g: (b, g_d)
        :return:
        '''

        output = self.operation_function(x, g)
        return output

    def _concatenation(self, x, g):
        input_size = x.size()
        batch_size = input_size[0]
        assert batch_size == g.size(0)

        # theta => (b, c, t, h, w) -> (b, i_c, t, h, w) -> (b, i_c, thw)
        # phi   => (b, g_d) -> (b, i_c)
        theta_x = self.theta(x)
        theta_x_size = theta_x.size()

        # g (b, c, t', h', w') -> phi_g (b, i_c, t', h', w')
        #  Relu(theta_x + phi_g + bias) -> f = (b, i_c, thw) -> (b, i_c, t/s1, h/s2, w/s3)
        phi_g = F.interpolate(self.phi(g), size=theta_x_size[2:], mode=self.upsample_mode)
        f = F.relu(theta_x + phi_g, inplace=True)

        #  psi^T * f -> (b, psi_i_c, t/s1, h/s2, w/s3)
        sigm_psi_f = torch.sigmoid(self.psi(f))

        # upsample the attentions and multiply
        sigm_psi_f = F.interpolate(sigm_psi_f, size=input_size[2:], mode=self.upsample_mode)
        y = sigm_psi_f.expand_as(x) * x
        W_y = self.W(y)

        return W_y, sigm_psi_f

class GridAttentionBlock2D(_GridAttentionBlockND):
    def __init__(self, in_channels, gating_channels, inter_channels=None, mode='concatenation',
                 sub_sample_factor=(2,2,2)):
        super(GridAttentionBlock2D, self).__init__(in_channels,
                                                   inter_channels=inter_channels,
                                                   gating_channels=gating_channels,
                                                   dimension=2, mode=mode,
                                                   sub_sample_factor=sub_sample_factor
                                                   )


class MultiAttentionBlock(nn.Module):
    def __init__(self, in_size, gate_size, inter_size, nonlocal_mode, sub_sample_factor):
        super(MultiAttentionBlock, self).__init__()
        self.gate_block_1 = GridAttentionBlock2D(in_channels=in_size, gating_channels=gate_size,
                                                 inter_channels=inter_size, mode=nonlocal_mode,
                                                 sub_sample_factor= sub_sample_factor)
        self.combine_gates = nn.Sequential(nn.Conv2d(in_size, in_size, kernel_size=1, stride=1, padding=0),
                                           nn.BatchNorm2d(in_size),
                                           nn.ReLU(inplace=True)
                                           )

        # initialise the blocks
        for m in self.children():
            if m.__class__.__name__.find('GridAttentionBlock2D') != -1: continue
            init_weights(m, init_type='kaiming')

    def forward(self, input, gating_signal):
        gate_1, attention_1 = self.gate_block_1(input, gating_signal)
        return self.combine_gates(gate_1), attention_1

In [0]:
class unet_simm(nn.Module):

    def __init__(self, feature_scale, n_classes, is_deconv, in_channels,
                 nonlocal_mode, attention_dsample, is_batchnorm):
        super(unet_simm, self).__init__()
        self.is_deconv = is_deconv
        self.in_channels = in_channels
        self.is_batchnorm = is_batchnorm
        self.feature_scale = feature_scale
        # self.use_cuda = Config.use_cuda

        filters = [64, 128, 256, 512, 1024]
        filters = [int(x / self.feature_scale) for x in filters]
        # filter [16, 32, 64, 128, 256]

        # downsampling
        self.conv1 = unetConv2(self.in_channels, filters[0], self.is_batchnorm, ks=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = unetConv2(filters[0], filters[1], self.is_batchnorm, ks=3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = unetConv2(filters[1], filters[2], self.is_batchnorm, ks=3)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)

        self.conv4 = unetConv2(filters[2], filters[3], self.is_batchnorm, ks=3)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2)

        self.center = unetConv2(filters[3], filters[4], self.is_batchnorm, ks=3)
        self.gating = UnetGridGatingSignal2(filters[4], filters[4], ks=1, is_batchnorm=self.is_batchnorm)

        # attention blocks
        self.attentionblock2 = MultiAttentionBlock(in_size=filters[1], gate_size=filters[2], inter_size=filters[1],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor= attention_dsample)
        self.attentionblock3 = MultiAttentionBlock(in_size=filters[2], gate_size=filters[3], inter_size=filters[2],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor= attention_dsample)
        self.attentionblock4 = MultiAttentionBlock(in_size=filters[3], gate_size=filters[4], inter_size=filters[3],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor= attention_dsample)

        # upsampling
        self.up_concat4 = unetUp2(filters[4], filters[3], False, is_batchnorm)
        # self.up_concat4 = unetUp2(512, filters[3], False, is_batchnorm)
        self.up_concat3 = unetUp2(filters[3], filters[2], False, is_batchnorm)
        self.up_concat2 = unetUp2(filters[2], filters[1], False, is_batchnorm)
        self.up_concat1 = unetUp2(filters[1], filters[0], False, is_batchnorm)

        # deep supervision
        self.dsv4 = UnetDsv2(in_size=filters[3], out_size=n_classes, scale_factor=8)
        self.dsv3 = UnetDsv2(in_size=filters[2], out_size=n_classes, scale_factor=4)
        self.dsv2 = UnetDsv2(in_size=filters[1], out_size=n_classes, scale_factor=2)
        self.dsv1 = nn.Conv2d(in_channels=filters[0], out_channels=n_classes, kernel_size=1)

        # final conv (without any concat)
        self.final = nn.Conv2d(n_classes*4, n_classes, 1)

        # initialise weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weights(m, init_type='kaiming')
            elif isinstance(m, nn.BatchNorm2d):
                init_weights(m, init_type='kaiming')

    def forward(self, inputs):

        # if self.use_cuda == False:
        #     # inputs = inputs.to(dtype=torch.float64)
        #     inputs = inputs.double()

        # inputs = inputs.double()

        # Feature Extraction
        conv1 = self.conv1(inputs)
        maxpool1 = self.maxpool1(conv1)

        conv2 = self.conv2(maxpool1)
        maxpool2 = self.maxpool2(conv2)

        conv3 = self.conv3(maxpool2)
        maxpool3 = self.maxpool3(conv3)

        conv4 = self.conv4(maxpool3)
        maxpool4 = self.maxpool4(conv4)

        # Gating Signal Generation
        center = self.center(maxpool4)
        gating = self.gating(center)

        # Attention Mechanism
        # Upscaling Part (Decoder)
        g_conv4, att4 = self.attentionblock4(conv4, gating)
        up4 = self.up_concat4(g_conv4, center)
        g_conv3, att3 = self.attentionblock3(conv3, up4)
        up3 = self.up_concat3(g_conv3, up4)
        g_conv2, att2 = self.attentionblock2(conv2, up3)
        up2 = self.up_concat2(g_conv2, up3)
        up1 = self.up_concat1(conv1, up2)

        # Deep Supervision
        dsv4 = self.dsv4(up4)
        dsv3 = self.dsv3(up3)
        dsv2 = self.dsv2(up2)
        dsv1 = self.dsv1(up1)
        final = self.final(torch.cat([dsv1,dsv2,dsv3,dsv4], dim=1))

        final = torch.sigmoid(final)

        return final

In [0]:
def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)
    
class SIMMDataset(Dataset):
    """SIMM dataset."""

    def __init__(self, root_dir, split, transform=None):
        """
        Args:
            dicomPaths (Array<string>): Array of DICOM file Paths.
            mask_csv_file (string): csv file with encoded masks (rle).
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.root_dir = root_dir
        self.dir_postfix = 'input/siim/'

        self.im_height = 1024
        self.im_width = 1024
        self.im_chan = 1

        ## Read masks file
        mask_csv_file = root_dir + 'masks/train-rle.csv' 
        print("Reading masks from: " + mask_csv_file)
        self.encodedMasks = pd.read_csv(mask_csv_file, names=['ImageId', 'EncodedPixels'], index_col='ImageId')

        ## Read dataset file names
        dsFile = root_dir + 'masks/simm_DS_' + split + '.csv'
        print("Reading ds from: " + dsFile)
        dsFileData = pd.read_csv(dsFile)
        self.dicomPaths = dsFileData['path'].tolist()
        # print("READ")
        self.transform = transform

    def __len__(self):
        return len(self.dicomPaths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Read dicom file
        dPath = self.dicomPaths[idx]
        dicom = pydicom.dcmread(self.root_dir + self.dir_postfix + dPath)
        
        # get image from dicom to numpy array
        image = np.array(dicom.pixel_array)
        
        # get mask (in rle) from csv
        landmarks = np.zeros((self.im_height, self.im_width), dtype=np.bool)
        
        fileId = dPath.split('/')[-1][:-4]
        rle = self.encodedMasks.loc[fileId, 'EncodedPixels']
        try:
            if type(rle) == str: # if single rle
                decodedRle = rle2mask(rle, self.im_height, self.im_width)
#                 landmarks = np.expand_dims(decodedRle, axis=2)
                landmarks = decodedRle
            else: # if multiple rle
                for x in rle:
                    decodedRle = rle2mask(x, self.im_height, self.im_width)
                    landmarks = landmarks + decodedRle
#                     landmarks = landmarks + np.expand_dims(decodedRle, axis=2)
        except Exception as e:
            print(e)
            
        #### TODO - IMPORTANT::: CHECK THIS  
        ## QUESTION: SHOULD WE TRANSPOSE THE MASK IN THE GETITEM FUNCTION 
        ## BECAUSE WHEN PLOTING THE GRAPHS WE HAVE TO TRANSPOSE IT.
        landmarks = landmarks.T

        # for some images, we have multiple masks, so we are adding the masks
        # which results in some pixels to > 1
        landmarks = (landmarks >= 1).astype('float64')

        if self.transform:
          # converting numpy array to PIL image
          # because torchvision transforms needs PIL image
          image = PIL.Image.fromarray(image)
          landmarks = PIL.Image.fromarray(landmarks)

          # apply transformation
          image, landmarks = self.transform(image, landmarks)

          # converting back to numpy array from PIL image
          image = np.array(image)
          landmarks = np.array(landmarks)

        img = np.expand_dims(image, axis=0)
        mk = np.expand_dims(landmarks, axis=0)

        # print("Final IMG:")
        # print(type(img))
        # print(img.shape)

        # normalize to 0-1
        img = img / 255

        return img, mk

In [0]:
class SIMMSoftDiceLoss(nn.Module):
    def __init__(self):
        super(SIMMSoftDiceLoss, self).__init__()

    def forward(self, input, target):
        smooth = 0.01
        batch_size = input.size(0)

        # input = torch.sigmoid(input).view(batch_size, -1)
        input = input.view(batch_size, -1)
        target = target.contiguous().view(batch_size, -1)

        inter = torch.sum(input * target, 1) + smooth
        union = torch.sum(input, 1) + torch.sum(target, 1) + smooth

        score = torch.sum(2.0 * inter / union, 0)
        score = 1.0 - score / float(batch_size)
        
        return score

In [0]:
def json_file_to_pyobj(jsonStr):
    def _json_object_hook(d): return collections.namedtuple('X', d.keys())(*d.values())
    def json2obj(data): return json.loads(data, object_hook=_json_object_hook)
    return json2obj(jsonStr)

In [0]:
def create_checkpoint(run_name, model, optimizer, is_best, epoch, network_options, train_loss, valid_loss, best_valid_loss):

  save_dir = os.path.join(MODEL_SAVE_DIR, run_name)
  if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)
    print('Creating save dir: ' + save_dir)

  print('Saving model checkpoint at epoch: ' + str(epoch))
  state = {
      'run_name': run_name,
      'model_state': model.state_dict(),
      'optimizer_state': optimizer.state_dict(),
      'epoch': epoch,
      'rng_state': torch.get_rng_state(),
      'network_options': network_options,
      'train_loss': train_loss,
      'valid_loss': valid_loss,
      'best_valid_loss': best_valid_loss
  }

  torch.save(state, os.path.join(MODEL_SAVE_DIR, run_name, 'checkpoint.pt'))
  if is_best:
    torch.save(state, os.path.join(MODEL_SAVE_DIR, run_name, 'checkpoint_best.pt'))

def load_checkpoint(run_name, model, optimizer):
  ckh_path = os.path.join(MODEL_SAVE_DIR, run_name, 'checkpoint.pt')
  checkpoint = torch.load(ckh_path)
  model.load_state_dict(checkpoint['model_state'])
  optimizer.load_state_dict(checkpoint['optimizer_state'])
  torch.set_rng_state(checkpoint['rng_state'])

  return model, optimizer, checkpoint

def print_checkpoint(checkpoint):
  print("Run Name: " + str(checkpoint['run_name']))
  print("Last epoch: " + str(checkpoint['epoch']))
  print("Train Loss: " + str(checkpoint['train_loss']))
  print("Validation Loss: " + str(checkpoint['valid_loss']))
  print("Best Validation Loss: " + str(checkpoint['best_valid_loss']))
  print("Options: " + str(checkpoint['network_options']))


Transformations

In [0]:
class SegCompose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class SegRandomAffine(object):
  def __init__(self, degrees, translate=None, scale=None, resample=False, fillcolor=0):
    # NOTE: shear is REMOVED FOR NOW

    # just to check the values using this contructor
    self.segTransform = T.RandomAffine(degrees, translate, scale, None, resample, fillcolor)

    if isinstance(degrees, numbers.Number):
        self.degrees = (-degrees, degrees)
    else:
        self.degrees = degrees
        
    self.translate = translate
    self.scale = scale
    self.resample = resample
    self.fillcolor = fillcolor

  def __call__(self, image, target):
        ret = T.RandomAffine.get_params(self.degrees, self.translate, self.scale, None, image.size)
        image = T.functional.affine(image, *ret, resample=self.resample, fillcolor=self.fillcolor)
        target = T.functional.affine(target, *ret, resample=self.resample, fillcolor=self.fillcolor)

        return image, target

class SegToTensor(object):
  def __init__(self):
    self.ToTensor = T.ToTensor()
  def __call__(self, image, target):
        image = self.ToTensor(image)
        target = self.ToTensor(target)

        return image, target

## Model Training

In [0]:
use_cuda = False
if torch.cuda.is_available():
  gpu_count = torch.cuda.device_count()
  print("Available GPU count:" + str(gpu_count))
  use_cuda = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
## Model Hyperparameters

NETWORK_OPTIONS = dict()
NETWORK_OPTIONS['USE_CUDA'] = use_cuda

# Set to false if only need to run on train dataset
NETWORK_OPTIONS['USE_VAL_SET'] = True

# NETWORK_OPTIONS['LR_POLICY'] = "step"
# NETWORK_OPTIONS['LR_DECAY_ITERS'] = 25
NETWORK_OPTIONS['LR_RATE'] = 1e-6
NETWORK_OPTIONS['L2_REG_WEIGHT'] = 1e-6

NETWORK_OPTIONS['BATCH_SIZE'] = 2

# dont forget to set run_name when continue_train is true
NETWORK_OPTIONS['CONTINUE_TRAIN'] = False
NETWORK_OPTIONS['RUN_NAME'] = ''
NETWORK_OPTIONS['START_EPOCH'] = 0
NETWORK_OPTIONS['NUM_EPOCHS'] = 100

NETWORK_OPTIONS['FEATURE_SCALE'] = 2
NETWORK_OPTIONS['DIVISION_FACTOR'] = 1

NETWORK_OPTIONS['DATALOADER_NUM_WORKERS'] = 1
NETWORK_OPTIONS['DATA_DIR'] = DATA_DIR
NETWORK_OPTIONS['TENSORBOARD_LOGS_DIR'] = TENSORBOARD_LOGS_DIR
NETWORK_OPTIONS['MODEL_SAVE_DIR'] = MODEL_SAVE_DIR

# converting dict() to python object (just for easy access)
NETWORK_OPTIONS_OBJ = json_file_to_pyobj(json.dumps(NETWORK_OPTIONS))

In [0]:
# Defining Dataloaders

rotate_val = 20.0
translation_val = [0.1,0.1]
scale_val = [0.7,1.3]

train_transform = SegCompose([
                              SegRandomAffine(degrees=rotate_val, translate=translation_val, scale=scale_val)
                             ])

train_dataset = SIMMDataset(NETWORK_OPTIONS_OBJ.DATA_DIR, split='train', transform=train_transform)
valid_dataset = SIMMDataset(NETWORK_OPTIONS_OBJ.DATA_DIR, split='validation', transform=None)
test_dataset  = SIMMDataset(NETWORK_OPTIONS_OBJ.DATA_DIR, split='test', transform=None)

train_loader = DataLoader(dataset=train_dataset, num_workers=NETWORK_OPTIONS_OBJ.DATALOADER_NUM_WORKERS, batch_size=NETWORK_OPTIONS_OBJ.BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, num_workers=NETWORK_OPTIONS_OBJ.DATALOADER_NUM_WORKERS, batch_size=NETWORK_OPTIONS_OBJ.BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(dataset=test_dataset,  num_workers=NETWORK_OPTIONS_OBJ.DATALOADER_NUM_WORKERS, batch_size=NETWORK_OPTIONS_OBJ.BATCH_SIZE, shuffle=False)

In [0]:
# Setup the NN Model
model = unet_simm(n_classes=1,
                      is_batchnorm=True,
                      in_channels=1,
                      nonlocal_mode='concatenation',
                      feature_scale=NETWORK_OPTIONS_OBJ.FEATURE_SCALE,
                      attention_dsample=(2,2),
                      is_deconv=False)

optimizer = optim.Adam(model.parameters(),
                        lr=NETWORK_OPTIONS_OBJ.LR_RATE,
                        betas=(0.9, 0.999),
                        weight_decay=NETWORK_OPTIONS_OBJ.L2_REG_WEIGHT)

# optimizer = optim.SGD(params,
#                               lr=option.lr_rate,
#                               momentum=0.9,
#                               nesterov=True,
#                               weight_decay=option.l2_reg_weight)

model = model.to(device)

In [0]:
last_best_valid_loss = 100.0
if NETWORK_OPTIONS_OBJ.CONTINUE_TRAIN == True:
  run_name = NETWORK_OPTIONS_OBJ.RUN_NAME
  model, optimizer, checkpoint = load_checkpoint(run_name, model, optimizer)
  last_best_valid_loss = checkpoint['best_valid_loss']
  print_checkpoint(checkpoint)

In [0]:
def get_images_grid(images):
  img_grid = torchvision.utils.make_grid(images, nrow=4)
  img_grid = img_grid.numpy()
  return np.transpose(img_grid, (1, 2, 0))

def show_image(images, labels):
  imgs = images.clone()

  imgs[labels == 1] = 255
  imgs[predictions >= 0.5] = 200

  imgs_grid = get_images_grid(imgs) 

  plt.imshow(imgs_grid, cmap=plt.cm.bone)
  plt.show()
  del imgs

def show_images_with_masks(images, labels):
  imgs_grid = get_images_grid(images)
  lbls_grid = get_images_grid(labels)

  plt.imshow(imgs_grid, cmap=plt.cm.bone)
  plt.imshow(lbls_grid, alpha=0.3, cmap="Reds")
  plt.show()

def show_images_with_masks_and_prediction(images, labels, predictions):
  imgs = images.clone()

  imgs[labels == 1] = 255
  imgs[predictions >= 0.5] = 200

  imgs_grid = get_images_grid(imgs) 

  plt.imshow(imgs_grid, cmap=plt.cm.bone)
  plt.show()
  del imgs

def matplotlib_imshow(img, one_channel=False):
  # if one_channel:
  #     img = img.mean(dim=0)
  # img = img / 2 + 0.5     # unnormalize
  npimg = img.numpy()
  if one_channel:
      plt.imshow(npimg, cmap=plt.cm.bone)
  else:
      plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(train_loader)
sample_images, sample_labels = dataiter.next()

ims = sample_images.clone()
ims[sample_labels == 1] = 255

# create grid of images
print(ims.shape)
img_grid = torchvision.utils.make_grid(ims)
print(img_grid.shape)

matplotlib_imshow(img_grid, one_channel=False)


In [0]:
%load_ext tensorboard
%tensorboard --logdir="$NETWORK_OPTIONS_OBJ.TENSORBOARD_LOGS_DIR"
# %reload_ext tensorboard
import time
time.sleep(10)

In [0]:
torch.cuda.empty_cache()

In [0]:
import time
timestr = time.strftime("%Y_%m_%d-%H_%M_%S")

run_name = timestr
if NETWORK_OPTIONS_OBJ.CONTINUE_TRAIN == True:
  run_name = NETWORK_OPTIONS_OBJ.RUN_NAME
print(run_name)

tb_log_dir = NETWORK_OPTIONS_OBJ.TENSORBOARD_LOGS_DIR + 'run_' + run_name
tb = SummaryWriter(flush_secs=10, log_dir=tb_log_dir)

criterion = SIMMSoftDiceLoss()
best_train_loss = 100.0
best_valid_loss = 100.0

if NETWORK_OPTIONS_OBJ.CONTINUE_TRAIN == False:
  tb.add_scalar("learning_rate", NETWORK_OPTIONS_OBJ.LR_RATE)
  tb.add_scalar("batch_size", NETWORK_OPTIONS_OBJ.BATCH_SIZE)
  tb.add_scalar("feature_scale", NETWORK_OPTIONS_OBJ.FEATURE_SCALE)
  tb.add_image('siim_sample_images', img_grid)
  tb.add_text('run_name', run_name)
  # tb.add_hparams(NETWORK_OPTIONS)
else:
  best_valid_loss = last_best_valid_loss


# tb.add_graph(model, sample_images)

tb.flush()

# Training loop
for epoch in range(NETWORK_OPTIONS_OBJ.START_EPOCH, NETWORK_OPTIONS_OBJ.START_EPOCH + NETWORK_OPTIONS_OBJ.NUM_EPOCHS):
  print('############# Running epoch: %d...\n' % (epoch))

  # Training Iterations
  running_loss = 0.0
  loss_per_epoch = 0.0
  epoch_batch_count = 0

  total_iter = total=len(train_loader)
  for epoch_iter, (images, labels) in tqdm(enumerate(train_loader, 1), total=total_iter):
    # Make a training update
    inputs = images.float().to(device)
    masks = labels.to(device)
    # assert input.size() == target.size()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, masks)

    # zero the parameter gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # update losses for epoch
    loss_per_epoch += loss.item()
    epoch_batch_count += 1
    # print statistics
    running_loss += loss.item()
    if epoch_iter % 20 == 0:
      #show_images_with_masks_and_prediction(images, labels, outputs)

      loss_avg = running_loss / 20
      #print('[%d, %5d] Running loss: %.3f' % (epoch + 1, epoch_iter + 1, loss_avg))
      # (10 * x + y) / 10
      i_num = (epoch + 1) + ((epoch_iter + 1) / total_iter)
      tb.add_scalar('RunningLoss', loss_avg, i_num)
      running_loss = 0.0

  loss = loss_per_epoch / epoch_batch_count
  tb.add_scalar('Loss', loss, epoch + 1)
  print('*********** [%d] Loss per epoch: %.3f' %(epoch + 1, loss))

  if loss <= best_train_loss:
    best_train_loss = loss

  # Validation Iterations
  running_loss_valid = 0.0
  loss_per_epoch_valid = 0.0
  epoch_batch_count_valid = 0
  loss_valid = 100
  if NETWORK_OPTIONS_OBJ.USE_VAL_SET:
    with torch.no_grad():
      for epoch_iter, (images, labels) in tqdm(enumerate(valid_loader, 1), total=len(valid_loader)):
        # get batch
        inputs = images.float().to(device)
        masks = labels.to(device)
        # assert input.size() == target.size()

        # forward
        outputs = model(inputs)
        loss = criterion(outputs, masks)

        # update losses for epoch
        loss_per_epoch_valid += loss.item()
        epoch_batch_count_valid += 1
        running_loss_valid += loss.item()

        if epoch_iter % 20 == 0:
          loss_avg = running_loss_valid / 20
          print('[%d, %5d] Running loss Validation: %.3f' % (epoch + 1, epoch_iter + 1, loss_avg))
          # (10 * x + y) / 10
          i_num = (10 * (epoch + 1) + (epoch_iter + 1)) - 10
          tb.add_scalar('RunningLossValidation', loss_avg, i_num)
          running_loss_valid = 0.0

    loss_valid = loss_per_epoch_valid / epoch_batch_count_valid
    tb.add_scalar('LossValidation', loss_valid, epoch + 1)
    print('*********** [%d] Validation Loss per epoch: %.3f' %(epoch + 1, loss_valid))


  is_best = False
  if loss_valid < best_valid_loss:
    best_valid_loss = loss_valid
    is_best = True

  create_checkpoint(run_name, model, optimizer, is_best, epoch, NETWORK_OPTIONS, loss, loss_valid, best_valid_loss)

  # reset loss per epoch
  loss_per_epoch = 0.0
  epoch_batch_count = 0
  # reset validation loss per epoch
  loss_per_epoch_valid = 0.0
  epoch_batch_count_valid = 0
  
  tb.flush()

  
  # Update the model learning rate
  # model.update_learning_rate()
tb.close()

In [0]:
# Calculate loss on test set
loss_test = 0.0
batch_count_test = 0

with torch.no_grad():
  for epoch_iter, (images, labels) in tqdm(enumerate(test_loader, 1), total=len(test_loader)):
    # get batch
    inputs = images.float().to(device)
    masks = labels.to(device)
    # assert input.size() == target.size()

    # forward
    outputs = model(inputs)
    loss = criterion(outputs, masks)

    # update losses for epoch
    loss_test += loss.item()
    batch_count_test += 1

loss_test = loss_test / batch_count_test
tb.add_scalar('LossTest', loss_test, loss_test)
print('\n Loss on Test Set: %.3f' %(loss_test))